In [26]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import copy

from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import export_graphviz
from sklearn.linear_model import Perceptron
from sklearn.dummy import DummyClassifier
from sklearn.metrics import roc_curve 
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsRegressor

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [37]:
def ams_score(ypred,y,weight):
    df = pd.concat([ypred,y,weight],axis=1)
    df.columns = ['ypred','y','weight']
    s = df[df['ypred']==1][df['y']==1]['weight'].sum()
    b = df[df['ypred']==1][df['y']==0]['weight'].sum()
    
    br = 10.0
    radicand = 2 *( (s+b+br) * np.log (1.0 + s/(b+br)) -s)
    if radicand < 0:
        print('radicand is negative. Exiting')
    else:
        return np.sqrt(radicand)

In [3]:
#Load database
df = pd.read_csv("atlas-higgs-challenge-2014-v2.csv")

#Traitement des valeurs manquantes
df= df.replace(-999, np.nan)
moy_col = df.mean()
for i in df.columns:
    if i not in ['EventId','KaggleWeight','Weight', "KaggleSet","Label"] :
        df[i] = df[i].replace(np.nan, moy_col[i])

In [7]:
#Creation du training set 
df_train = df[df['KaggleSet']=='t']
x_train = copy.copy(df_train).drop(
    ['EventId','KaggleWeight','Weight', "KaggleSet","Label"], axis =1)
y_train = copy.copy(df_train['Label'])
y_train = y_train.replace('s',1)
y_train = y_train.replace('b',0)

#Création testing set
df_test = df[df['KaggleSet']!='t']
df_test = df_test[df_test['KaggleSet']!='u']
x_test = copy.copy(df_test).drop(
    ['EventId','KaggleWeight','Weight', "KaggleSet","Label"], axis =1)
y_test = copy.copy(df_test['Label'])
y_test = y_test.replace('s',1)
y_test = y_test.replace('b',0)

#Création pour score AMS
'''Solution File header: EventId, Class, Weight
Submission File header: EventId, RankOrder, Class'''

df_solution = pd.DataFrame()
df_solution['EventId'] = df_test['EventId']
df_solution['Label'] = df_test['Label']
df_solution['Weight'] = df_test['Weight']
df_solution.to_csv('solution', index=False)

df_submission = pd.DataFrame()
df_submission['EventId'] = df_test['EventId']
df_submission['RankOrder'] = df_test['EventId'] #pas nutilisé par AMS


In [38]:
#Base line par caractéristique
accuracy_train_list = []
accuracy_test_list = []
ams_list = []
for col in x_train.columns:
    #crée les nouveaux sets
    x_train_col = x_train[col].to_frame()
    x_test_col = x_test[col].to_frame()
    
    clf = DecisionTreeClassifier(max_depth = 10)
    clf.fit(x_train_col,y_train)
    ypred = pd.DataFrame(clf.predict(x_test_col),index = y_test.index)
    ams_list.append(ams_score(ypred,y_test,df_test['Weight']))
    accuracy_train_list.append(clf.score(x_train_col,y_train))
    accuracy_test_list.append(clf.score(x_test_col,y_test))
    
#Sauvegarde des scores
df_scores = pd.DataFrame([accuracy_train_list,accuracy_test_list,ams_list],
                         columns= x_train.columns,
                         index = ['train_acc','test_acc','ams'])
writer = pd.ExcelWriter('baseline_caracteristique.xlsx')
df_scores.to_excel(writer,'Sheet1')
writer.save()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  after removing the cwd from sys.path.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """


In [39]:
#Baseline toutes les caractéristiques 
clf = DecisionTreeClassifier(max_depth = 10)
clf.fit(x_train,y_train)
ypred = pd.DataFrame(clf.predict(x_test),index = y_test.index)
print(ams_score(ypred,y_test,df_test['Weight']))
print(clf.score(x_train,y_train))
print(clf.score(x_test,y_test))


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  after removing the cwd from sys.path.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """


2.102925949182895
0.834736
0.8218563636363636


In [41]:
#Baseline avec toutes les caractéristique sans jouer sur les paramètres 
accuracy_train_list = []
accuracy_test_list = []
ams_list = []
list_algo = [GradientBoostingClassifier(),
          RandomForestClassifier(),
          KNeighborsRegressor()]
for clf in list_algo:
    clf.fit(x_train,y_train)
    ypred = pd.DataFrame(clf.predict(x_test),index = y_test.index)
    ams_list.append(ams_score(ypred,y_test,df_test['Weight']))
    accuracy_train_list.append(clf.score(x_train,y_train))
    accuracy_test_list.append(clf.score(x_test,y_test))
    
#Sauvegarde des scores
df_scores = pd.DataFrame([accuracy_train_list,accuracy_test_list,ams_list],
                         columns= ['grad_boost','random_forest','k_neighbors'],
                        index = ['train_acc','test_acc','ams'])
writer = pd.ExcelWriter('naive_results.xlsx')
df_scores.to_excel(writer,'Sheet1')
writer.save()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  after removing the cwd from sys.path.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  after removing the cwd from sys.path.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """


In [ ]:
#Baseline avec toutes les caractéristique en jouant sur les paramètres 
accuracy_train_list = []
accuracy_test_list = []
ams_list = []
list_algo = [GradientBoostingClassifier(loss = 'exponential'),
          RandomForestClassifier(n_estimators=20),
          RandomForestClassifier(n_estimators=30,max_depth = 15),
          KNeighborsRegressor(weights = 'distance'),
          KNeighborsRegressor(weights = 'distance', n_neighbors = 10),
          KNeighborsRegressor(n_neighbors = 10)]
for clf in list_algo:
    clf.fit(x_train,y_train)
    ypred = pd.DataFrame(clf.predict(x_test),index = y_test.index)
    ams_list.append(ams_score(ypred,y_test,df_test['Weight']))
    accuracy_train_list.append(clf.score(x_train,y_train))
    accuracy_test_list.append(clf.score(x_test,y_test))
    
#Sauvegarde des scores
df_scores = pd.DataFrame([accuracy_train_list,accuracy_test_list,ams_list],
                         columns= ['grad_boost[adaboost]','random_forest[20]','random_forest[30,15]',
                                   'k_neighbors[distance]','k_neighbors[distance,10]',
                                   'k_neighbors[10]'],
                        index = ['train_acc','test_acc','ams'])
writer = pd.ExcelWriter('param_results_v1.xlsx')
df_scores.to_excel(writer,'Sheet1')
writer.save()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  after removing the cwd from sys.path.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  after removing the cwd from sys.path.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  after removing the cwd from sys.path.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  ""

In [47]:
ams_list

[2.233902294077339,
 2.2943544345009306,
 2.336065680545118,
 1.794332094947172,
 1.6142495525945528]

In [ ]:
KNeighborsRegressor:
   - n_neighbors: (5) plus proche voisin
   - weight: (uniform) tester avec 'distance'

RandomForestClassifier:
    - n_estimators: (10) nombres d'arbre dans la foret,
    - max_depth: (None) profondeur max des arbres

GradientBoostingClassifier:
    - loss: (deviance): tester exponential (adaboost)
    - n_estimators: (100) 